In [1]:
%matplotlib inline 

from collections import defaultdict

import pandas as pd
import matplotlib.pyplot as plt
import requests
from pattern import web

In [2]:
url = 'http://en.wikipedia.org/wiki/List_of_countries_by_past_and_future_population'
website_html = requests.get(url).text

In [3]:
def get_population_html(html):
    dom=web.Element(html)
    tbls=dom.by_class("sortable wikitable")
    return tbls

tables=get_population_html(website_html)
print "Table length is: %d" % len(tables)
for t in tables: 
    print t.attributes
print tables   

Table length is: 6
{u'style': u'text-align: right', u'class': u'sortable wikitable'}
{u'style': u'text-align: right', u'class': u'sortable wikitable'}
{u'style': u'text-align: right', u'class': u'sortable wikitable'}
{u'style': u'text-align: right', u'class': u'sortable wikitable'}
{u'style': u'text-align: right', u'class': u'sortable wikitable'}
{u'style': u'text-align: right', u'class': u'sortable wikitable'}
[Element(tag=u'table'), Element(tag=u'table'), Element(tag=u'table'), Element(tag=u'table'), Element(tag=u'table'), Element(tag=u'table')]


In [4]:
for tbl in tables:
    print tbl('th'
              )[0].content
    

Country or territory
(Sub)continent
Country or territory
(Sub)continent
Country or territory
(Sub)continent


In [5]:
def table_type(tbl):
    
    return tbl('th')[0].content

tables_by_type=defaultdict(list)
for tbl in tables:
    tables_by_type[table_type(tbl)].append(tbl)
print tables_by_type   

defaultdict(<type 'list'>, {u'Country or territory': [Element(tag=u'table'), Element(tag=u'table'), Element(tag=u'table')], u'(Sub)continent': [Element(tag=u'table'), Element(tag=u'table'), Element(tag=u'table')]})


In [9]:
def get_countries_population(tables):
    result=defaultdict(dict)
    for tbl in tables:
        headers=tbl('tr')
        first_header=headers[0]
        th_s=first_header('th')
        years=[int(val.content) for val in th_s if val.content.isnumeric()]
        year_indices=[idx for idx,val in enumerate(th_s) if val.content.isnumeric()]
        print years 
        print year_indices
        
        
        rows=tbl('tr')[1:]
        for row in rows:
            tds=row('td')
            country_name=tds[0]('a')[0].content
            population_by_year=[int(tds[colidx].content.replace(",","")) for colidx in year_indices]
            subdict=dict(zip(years,population_by_year))
            result[country_name].update(subdict)
            
            
            
            
    return result
result=get_countries_population(tables_by_type[u'Country or territory'])
print len(result), "Countries extracted"
print result[u'Canada']

[1950, 1955, 1960, 1965, 1970, 1975, 1980]
[1, 2, 4, 6, 8, 10, 12]
[1985, 1990, 1995, 2000, 2005, 2010, 2015]
[1, 3, 5, 7, 9, 11, 13]
[2020, 2025, 2030, 2035, 2040, 2045, 2050]
[1, 3, 5, 7, 9, 11, 13]
227 Countries extracted
{1985: 25942, 2050: 41136, 1955: 16050, 2020: 36387, 1990: 27791, 1960: 18267, 2025: 37559, 1995: 29691, 1965: 20071, 2030: 38565, 2000: 31100, 1970: 21750, 2035: 39396, 2005: 32386, 1975: 23209, 2040: 40070, 2010: 33760, 1980: 24593, 2045: 40635, 1950: 14011, 2015: 35100}


In [11]:
df=pd.DataFrame.from_dict(result,orient='index')
#sort based on the years
df.sort(axis=1,inplace=True)
print df
#Finally, we obtain the data in a clean format

                                1950    1955    1960    1965    1970    1975  \
Afghanistan                     8150    8891    9829   10998   12431   14132   
Albania                         1227    1392    1623    1884    2157    2401   
Algeria                         8893    9842   10909   11963   13932   16140   
American Samoa                    19      20      20      25      27      30   
Andorra                            6       6       8      14      20      27   
Angola                          4118    4423    4797    5135    5606    6050   
Anguilla                           5       5       6       6       6       6   
Antigua and Barbuda               46      51      55      59      66      68   
Argentina                      17150   18928   20616   22283   23962   26082   
Armenia                         1355    1565    1869    2206    2520    2834   
Aruba                             50      54      57      59      59      59   
Australia                       8267    

In [20]:
print df.ix["India"]

1950     369880
1955     404268
1960     445393
1965     494964
1970     553889
1975     618923
1980     684888
1985     759612
1990     838159
1995     920585
2000    1006300
2005    1090973
2010    1173108
2015    1251696
2020    1326093
2025    1396046
2030    1460743
2035    1519491
2040    1571715
2045    1617238
2050    1656554
Name: India, dtype: int64
